In [1]:
# gen arg

import os
import re
import json
import copy
import time
import random
import pickle

from utils.chat2DeepSeek import Chat2DeepSeek
from utils.gen_prompt import Prompt
from utils.util import parse_trigger_answer, parse_argument_answer, save_json, load_json

In [2]:
API_KEY = 'sk-8b0397e1abf144c38469970c33a06ba4'

In [3]:
event_dict = load_json('./meta_data/event_dict_full.json')

In [4]:
n_event = 3
n_trigger = 100
n_argument = 20
complex_score = [5, 10]
max_argument = 4


In [ ]:
event_arg = {}
event_tri = {}

In [5]:
def get_local_time():
    return str(time.localtime().tm_mon) + '_' + str(time.localtime().tm_mday) + '_' + str(time.localtime().tm_hour) + '_' + str(time.localtime().tm_min)

In [6]:
def chat4argument_pool(gen_prompt, n_argument, event_type, event_def, role_dict):
    chat = Chat2DeepSeek(api_key=API_KEY)
    response = chat.prompt2chat(gen_prompt.gen_argument_promt(n_argument, event_type, event_def, role_dict)).strip()
    a_d = re.sub('^```json', '', response, flags=re.I)
    a_d = re.sub('```$', '', a_d, flags=re.I)
    try:
        return json.loads(a_d)
    except:
        return a_d

In [7]:
def chat4argument_pool_all(event_dict, n_argument):
    for i, event in enumerate(event_dict, 1):
        # event_dict = {'ATTACK':[event_def, role_dict, example, exam_trigger]}
        try:
            if event_arg.get(event):
                print('---EVENT {} chated'.format(event))
                continue
            gen_prompt = Prompt()
            event_type = event
            event_def = event_dict[event][0]
            role_dict = event_dict[event][1]
    
            start_time = time.time()
            print('TIME {} -- chating for EVENT {}'.format(get_local_time(), event))
            a_d = chat4argument_pool(gen_prompt, n_argument, event_type, event_def, role_dict)
            print('TIME {} -- done chating'.format(get_local_time()))
            if type(a_d) == type({}):
                event_arg[event] = a_d
            else:
                print('error ', event)
                try:
                    with open('./log/error/' + event + '_arg.txt', 'w') as f:
                        f.write(a_d)
                except:
                    print('---can not save:', a_d)
            end_time = time.time()
            print('Done processing EVENT: {}, in {} s'.format(event, end_time - start_time))
            time.sleep(5)
        except:
            continue
        save_json(event_tri, './arg_{}_{}_{}.json'.format(i, event, get_local_time()))
    print('ALL DONE')

In [ ]:
chat4argument_pool_all(event_dict, n_argument)

TIME 3_2_22_14 -- chating for EVENT START-ORG
[{'role': 'system', 'content': 'An expert in Event Extraction of NLP'}, {'role': 'user', 'content': '\tGiven event type START-ORG and its definition:\n\tSTART-ORG: A START-ORG Event occurs whenever a new ORGANIZATION is created.\n\tPlease generate arguments corresponding to the event type START-ORG and the definitions of each argument role given below (argument should be classified to one argument role only). That is, to answer those questions:\n\t1.Given the definition of Agent argument role as ‘The agent responsible for the START-ORG Event (the ‘founder’)’, what are some possible Person/Organization/Geo-Political names that can be used as Agent?\n\t2.Given the definition of Org argument role as ‘The ORGANIZATION that is started’, what are some possible Organization names that can be used as Org?\n\t3.Given the definition of Time argument role as ‘When the Event takes place’, what are some possible Time names that can be used as Time?\n\t4